In [1]:
# 부산버스정류장
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from datetime import datetime
from ServiceKey import public_serviceKey,mySQL_Connect

mydb = mySQL_Connect()

cursor = mydb.cursor(prepared=True)
# 2021.09.21일 API오류로 인한 수집 실패
query = ("select bstopId from public_busanbusstts where 6658 <= seq and seq <= 7600")

cursor.execute(query)

location_list = []

for i in cursor:
    location_list.append(i)
    
now = datetime.now()
nowDate = now.strftime('%Y%m%d')

cnt = 1
for i in location_list:
    bstopId = i
    print("조회날짜 : {} 순번 : {} start {} public_busanbusstts".format(nowDate,cnt,bstopId[0]))
    
    url = 'http://61.43.246.153/openapi-data/service/busanBIMS2/stopArr'
    queryParams = '?' + urlencode({
        quote_plus('serviceKey') : public_serviceKey(),
        quote_plus('bstopid') : bstopId[0]
    })

    response = requests.get(url + queryParams).text.encode('utf-8')
    xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

    rows = xmlobj.findAll('item')
    resultCode = xmlobj.findAll('resultCode')

    if resultCode[0].text == '99':
        print("{} 번째부터 조회수 제한이 걸렸습니다.".format(bstopId))
        mydb.close()
        break
    cnt += 1
    for j in rows:
        item = j
        arsNo = item.findAll('arsNo')
        bstopId = item.findAll('bstopId')
        bstopIdx = item.findAll('bstopIdx')
        bustype = item.findAll('bustype')
        lng = item.findAll('gpsX')
        lat = item.findAll('gpsY')
        lineNo = item.findAll('lineNo')
        lineid = item.findAll('lineid')
        nodeNm = item.findAll('nodeNm')
        
        try:
            sql = "INSERT INTO travel_supplies_in_pockets.public_busanbuslineid_info (arsNo, bstopId, bstopIdx, bustype, lat, lng, lineNo, lineid, nodeNm) VALUES (?,?,?,?,?,?,?,?,?)"
            val = (arsNo[0].text, bstopId[0].text, bstopIdx[0].text, bustype[0].text, lat[0].text, lng[0].text, lineNo[0].text, lineid[0].text, nodeNm[0].text)
        except IndexError:
            print(item)
            sql = "INSERT INTO travel_supplies_in_pockets.public_busanbuslineid_info (arsNo, bstopId, bstopIdx, bustype, lat, lng, lineNo, lineid, nodeNm) VALUES (?,?,?,?,?,?,?,?,?)"
            val = (arsNo[0].text, bstopId[0].text, bstopIdx[0].text, bustype[0].text, lat[0].text, lng[0].text, lineNo[0].text, lineid[0].text, nodeNm[0].text)
            
        cursor.execute(sql, val)
        mydb.commit()

조회날짜 : 20210922 순번 : 1 start 186200204 public_busanbusstts
(186200204,) 번째부터 조회수 제한이 걸렸습니다.
